In [1]:
import os
import pickle
import pandas as pd
from tqdm import tqdm 

DATA_DIR = "data/wiki-join-jaccard/"
DATALAKE_DIR = os.path.join(DATA_DIR, "datalake")
BENCHMARK_FILE = os.path.join(DATA_DIR, "benchmark.pkl")

# Step 1: Load benchmark.pkl
with open(BENCHMARK_FILE, 'rb') as f:
    benchmark_data = pickle.load(f)

# Step 2: Read all CSV files into a dictionary
# We will map (csv_filename) -> { column_name -> list_of_values }
all_tables = {}

for csv_filename in tqdm(os.listdir(DATALAKE_DIR)):
    if not csv_filename.endswith('.csv'):
        continue
    path = os.path.join(DATALAKE_DIR, csv_filename)
    try:
        df = pd.read_csv(path, dtype=str, on_bad_lines='skip')  # read as strings
        df = df.fillna('')                # replace NaNs with empty strings
        table_dict = {}
        for col in df.columns:
            col_vals = df[col].unique().tolist()
            # Remove duplicates and sanitize
            col_vals = [val.strip() for val in col_vals if val.strip()]
            table_dict[col] = col_vals
        all_tables[csv_filename] = table_dict
    except Exception as e:
        print(f"Error reading {csv_filename}: {e}")


  8%|███████████████████▌                                                                                                                                                                                                                     | 3917/46521 [00:21<03:56, 180.30it/s]


KeyboardInterrupt: 

In [ ]:
import os
from tqdm import tqdm

# Build the list of columns from all_tables
columns = []
for csv_file, table_dict in tqdm(all_tables.items()):
    table_title = os.path.splitext(csv_file)[0]  # Remove the .csv extension
    for col_name, values in table_dict.items():
        columns.append({
            "table_title": table_title,
            "column_name": col_name,
            "values": values
        })

print(f"Number of columns: {len(columns)}")


In [ ]:
import math
import nltk
from collections import Counter

def column_to_text(table_title: str, column_name: str, col_vals, max_tokens=128) -> str:
    """
    Creates a text summary for a column.
    
    The summary includes the column name, number of distinct (non-empty) values,
    basic statistics (max, min, average length), and a comma-separated list of
    unique values sorted by descending TF-IDF computed locally per column.
    
    Args:
      table_title (str): Title of the table (e.g., derived from the CSV filename).
      column_name (str): The name of the column.
      col_vals (list): List of cell values (strings).
      max_tokens (int): Maximum number of tokens in the final summary.
    
    Returns:
      A truncated text summary string.
    """
    # Filter out empty strings and ensure values are strings
    filtered_vals = [str(v).strip() for v in col_vals if str(v).strip() != ""]
    total_rows = len(filtered_vals)
    if total_rows == 0:
        return ""
    
    # Count frequency of each unique value
    value_counts = Counter(filtered_vals)
    
    # Compute local TF-IDF for each unique value
    tfidf_scores = {}
    for value, count in value_counts.items():
        tf = count / total_rows
        idf = math.log((total_rows + 1) / (count + 1)) + 1
        tfidf_scores[value] = tf * idf
    
    # Sort values by descending TF-IDF score
    sorted_values = sorted(tfidf_scores, key=lambda v: tfidf_scores[v], reverse=True)
    
    n_distinct = len(sorted_values)
    lengths = [len(v) for v in sorted_values]
    max_len = max(lengths) if lengths else 0
    min_len = min(lengths) if lengths else 0
    avg_len = sum(lengths) / len(lengths) if lengths else 0
    
    col_str = ", ".join(sorted_values)
    
    summary = (f"{table_title}. {column_name} contains {n_distinct} values "
               f"({max_len}, {min_len}, {avg_len:.1f}): {col_str}")
    
    # Tokenize and truncate
    tokens = nltk.word_tokenize(summary)
    truncated_tokens = tokens[:int(max_tokens)]
    truncated_sentence = " ".join(truncated_tokens)
    return truncated_sentence


In [ ]:
from sentence_transformers import SentenceTransformer, losses
from torch.utils.data import DataLoader
import faiss
import numpy as np




In [ ]:
# For each column, compute its text summary.
corpus_texts = []
id_to_metadata = []  # To map index IDs back to (csv_filename, column_name)
for col in tqdm(columns):
    text = column_to_text(col["table_title"], col["column_name"], col["values"], max_tokens=128)
    corpus_texts.append(text)
    # Reconstruct CSV filename from table title (append .csv)
    csv_filename = col["table_title"] + ".csv"
    id_to_metadata.append((csv_filename, col["column_name"]))

print(f"Total corpus texts: {len(corpus_texts)}")

In [ ]:
corpus_texts[0]

# MPNET

In [ ]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
corpus_embeddings = model.encode(corpus_texts, convert_to_numpy=True, show_progress_bar=True)

d = corpus_embeddings.shape[1]
# Using a simple FlatL2 index here for max accuracy; for larger datasets you may choose HNSW or IVF-PQ.
index = faiss.IndexFlatL2(d)
index.add(corpus_embeddings)
print(f"FAISS index built with {index.ntotal} entries.")

In [ ]:
import os
import random
import json
import numpy as np
from tqdm import tqdm

def evaluate_inference_to_json(model, index, id_to_metadata, benchmark_data, all_tables,
                               n_splits=5, split_frac=0.01, max_k=10, max_tokens=128,
                               output_json_file="evaluation_results.json"):
    """
    Evaluate inference using n_splits random subsets of the benchmark.
    For each split, sample split_frac fraction of the benchmark queries.
    For each query and for k = 1 to max_k, compute precision, recall, F1 and average precision (ap).
    Aggregates per-query metrics into system-level metrics and saves both into a JSON file.
    
    Args:
      model: A SentenceTransformer model.
      index: A FAISS index built over the corpus embeddings.
      id_to_metadata: List mapping index IDs to (csv_filename, column_name).
      benchmark_data: dict mapping (csv_filename, column_name) -> list of joinable columns.
      all_tables: dict mapping csv_filename to a dict of {column_name: column_values} (used in column_to_text).
      n_splits (int): Number of random splits.
      split_frac (float): Fraction of benchmark queries to sample in each split.
      max_k (int): Maximum k value for computing metrics.
      max_tokens (int): Maximum tokens to use in the column_to_text conversion.
      output_json_file (str): Path to the JSON file where results will be saved.
    
    Returns:
      results (dict): Dictionary with system-level and per-query metrics.
    """
    benchmark_keys = list(benchmark_data.keys())
    num_queries = len(benchmark_keys)
    
    # Accumulators for system metrics over all evaluated queries
    system_precision = np.zeros(max_k)
    system_recall    = np.zeros(max_k)
    system_ap        = np.zeros(max_k)
    f1_lists = [[] for _ in range(max_k)]
    
    # Dictionary to hold per-query results.
    per_query_metrics = {}
    total_evaluated_queries = 0
    
    for split in range(n_splits):
        random.seed(1000 + split)
        sample_size = max(1, int(split_frac * num_queries))
        sampled_keys = random.sample(benchmark_keys, sample_size)
        
        for query_key in tqdm(sampled_keys, desc=f"Split {split+1}/{n_splits}"):
            csv_filename, col_name = query_key
            # Skip if the query column is not available
            if csv_filename not in all_tables or col_name not in all_tables[csv_filename]:
                continue
            
            table_title = os.path.splitext(csv_filename)[0]
            # Assume column_to_text is defined elsewhere
            query_text = column_to_text(table_title, col_name, all_tables[csv_filename][col_name],
                                        max_tokens=max_tokens)
            query_embedding = model.encode([query_text], convert_to_numpy=True)
            distances, indices = index.search(query_embedding, max_k)
            
            # Retrieved candidate list (as given by the index)
            retrieved = [id_to_metadata[i] for i in indices[0]]
            
            # Ground truth set for this query
            gt_set = set(benchmark_data[query_key])
            total_relevant = len(gt_set)
            
            # Initialize per-query metric lists
            query_metrics = {
                'candidates': retrieved,
                'ground_truth': list(gt_set),
                'precision': [],
                'recall': [],
                'f1': [],
                'ap': []
            }
            
            for k in range(1, max_k+1):
                retrieved_k = retrieved[:k]
                # Compute precision and recall
                num_relevant = sum(1 for item in retrieved_k if item in gt_set)
                precision = num_relevant / k
                recall = num_relevant / total_relevant if total_relevant > 0 else 0.0
                # F1 score
                f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0
                
                # Compute average precision (AP) for top-k
                ap = 0.0
                relevant_count = 0
                for i, item in enumerate(retrieved_k, start=1):
                    if item in gt_set:
                        relevant_count += 1
                        ap += relevant_count / i
                norm = min(total_relevant, k)
                ap = ap / norm if norm > 0 else 0.0
                
                query_metrics['precision'].append(precision)
                query_metrics['recall'].append(recall)
                query_metrics['f1'].append(f1)
                query_metrics['ap'].append(ap)
                
                # Accumulate system-level metrics
                system_precision[k-1] += precision
                system_recall[k-1]    += recall
                system_ap[k-1]        += ap
                f1_lists[k-1].append(f1)
            
            # Use a unique key per query that also encodes the split
            query_key_str = f"split_{split}_{csv_filename}::{col_name}"
            per_query_metrics[query_key_str] = query_metrics
            total_evaluated_queries += 1

    # Average system metrics over all queries
    if total_evaluated_queries > 0:
        avg_system_precision = (system_precision / total_evaluated_queries).tolist()
        avg_system_recall = (system_recall / total_evaluated_queries).tolist()
        avg_system_ap = (system_ap / total_evaluated_queries).tolist()
        avg_system_f1 = [np.mean(f1_list) if f1_list else 0.0 for f1_list in f1_lists]
    else:
        avg_system_precision = [0.0] * max_k
        avg_system_recall = [0.0] * max_k
        avg_system_ap = [0.0] * max_k
        avg_system_f1 = [0.0] * max_k
    
    used_k = list(range(1, max_k+1))
    metrics_at_k = {
        k: {
            'precision': avg_system_precision[k-1],
            'recall': avg_system_recall[k-1],
            'f1': avg_system_f1[k-1],
            'map': avg_system_ap[k-1]
        }
        for k in used_k
    }
    
    results = {
        'system_metrics': {
            'precision': avg_system_precision,
            'recall': avg_system_recall,
            'f1': avg_system_f1,
            'map': avg_system_ap,
            'used_k': used_k,
            'metrics_at_k': metrics_at_k
        },
        'per_query_metrics': per_query_metrics
    }

    # if output path doesn't exist, create it
    os.makedirs(os.path.dirname(output_json_file), exist_ok=True)
    
    # Save the results to a JSON file.
    with open(output_json_file, "w") as f:
        json.dump(results, f, indent=2)
    
    print(f"Results saved to {output_json_file}")
    return results

In [ ]:
# Set the number of top results to retrieve (e.g., top 10)
model_name = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(model_name)
results = evaluate_inference_to_json(model, index, id_to_metadata, 
                                                benchmark_data, all_tables, n_splits=1, split_frac=1, 
                                                max_k=10, max_tokens=128, output_json_file="output/wiki-join-jaccard/deepjoin/deepjoin_mpnet.json")


# Fasttext

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors

def load_fasttext_model():
    """Load and return FastText model using gensim downloader (caching it locally)."""
    model_path = 'checkpoints/fasttext/fasttext-wiki-news-subwords-300'
    if os.path.exists(model_path):
        model = KeyedVectors.load(model_path)
    else:
        model = api.load("fasttext-wiki-news-subwords-300")
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        model.save(model_path)
    return model

fasttext_model = load_fasttext_model()

In [ ]:
import numpy as np
import nltk
from tqdm import tqdm

def embed_text_with_fasttext(text, fasttext_model):
    """
    Tokenizes the text and returns the average FastText embedding.
    If no tokens are found in the FastText vocabulary, returns a zero vector.
    
    Args:
      text (str): The text summary of the column.
      fasttext_model: A pre-loaded FastText model.
      
    Returns:
      np.array: A vector representing the embedding of the text.
    """
    # Tokenize the text (ensure you have downloaded punkt from nltk)
    tokens = nltk.word_tokenize(text)
    
    # Retrieve embeddings for tokens found in the model's vocabulary.
    token_vectors = [fasttext_model[word] for word in tokens if word in fasttext_model]
    
    if not token_vectors:
        # Return a zero vector if no token embeddings are found
        return np.zeros(fasttext_model.vector_size)
    else:
        # Average the token vectors to get a single embedding for the column
        return np.mean(token_vectors, axis=0)


# Compute embeddings for each column text in corpus_texts
column_embeddings = []
for text in tqdm(corpus_texts, desc="Embedding columns"):
    emb = embed_text_with_fasttext(text, fasttext_model)
    column_embeddings.append(emb)

print(f"Computed embeddings for {len(column_embeddings)} columns.")


In [ ]:
import faiss
import numpy as np

# Assume column_embeddings is a list of numpy arrays (each is a fastText embedding)
corpus_embeddings = np.vstack(column_embeddings)  # shape: (num_columns, vector_dim)

d = corpus_embeddings.shape[1]  # dimension of embeddings

# Create a simple index using L2 distance.
index = faiss.IndexFlatL2(d)
index.add(corpus_embeddings)
print(f"FAISS index built with {index.ntotal} entries.")


In [ ]:
import os
import random
import json
import numpy as np
from tqdm import tqdm

def evaluate_inference_fasttext_to_json(fasttext_model, index, id_to_metadata, benchmark_data, all_tables,
                                        n_splits=5, split_frac=0.01, max_k=10, max_tokens=128,
                                        output_json_file="evaluation_fasttext_results.json"):
    """
    Evaluate inference using n_splits random subsets of the benchmark with FastText embeddings.
    For each split, a fraction of the benchmark queries are sampled. For each query and for k = 1 to max_k,
    precision, recall, F1 and average precision (AP) are computed.
    
    System-level metrics (averaged over all queries) and per-query metrics are aggregated and saved into a JSON file.
    
    Args:
      fasttext_model: A FastText model (gensim KeyedVectors) loaded previously.
      index: A FAISS index built over the corpus embeddings.
      id_to_metadata: List mapping index IDs to (csv_filename, column_name).
      benchmark_data: dict mapping (csv_filename, column_name) -> list of joinable columns.
      all_tables: dict mapping csv_filename to a dict of {column_name: column_values} (used in column_to_text).
      n_splits (int): Number of random splits.
      split_frac (float): Fraction of benchmark queries to sample in each split.
      max_k (int): Maximum k value for computing metrics.
      max_tokens (int): Maximum tokens to use in the column_to_text conversion.
      output_json_file (str): Path to the JSON file where results will be saved.
    
    Returns:
      results (dict): Dictionary with system-level and per-query metrics.
    """
    benchmark_keys = list(benchmark_data.keys())
    num_queries = len(benchmark_keys)
    
    # Accumulators for system metrics over all evaluated queries.
    system_precision = np.zeros(max_k)
    system_recall    = np.zeros(max_k)
    system_ap        = np.zeros(max_k)
    f1_lists = [[] for _ in range(max_k)]
    
    # Dictionary to hold per-query metrics.
    per_query_metrics = {}
    total_evaluated_queries = 0
    
    for split in range(n_splits):
        # Fix the random seed for reproducibility.
        random.seed(1000 + split)
        sample_size = max(1, int(split_frac * num_queries))
        sampled_keys = random.sample(benchmark_keys, sample_size)
        
        for query_key in tqdm(sampled_keys, desc=f"Split {split+1}/{n_splits}"):
            csv_filename, col_name = query_key
            # Check if the column exists in our datalake.
            if csv_filename not in all_tables or col_name not in all_tables[csv_filename]:
                continue
            
            table_title = os.path.splitext(csv_filename)[0]
            # Convert the column to text (assuming column_to_text is defined elsewhere).
            query_text = column_to_text(table_title, col_name, all_tables[csv_filename][col_name],
                                        max_tokens=max_tokens)
            # Use FastText to embed the query text.
            query_emb = embed_text_with_fasttext(query_text, fasttext_model)
            # FAISS expects a 2D array.
            query_embedding = query_emb.reshape(1, -1)
            
            distances, indices = index.search(query_embedding, max_k)
            # Build the list of retrieved candidates.
            retrieved = [id_to_metadata[idx] for idx in indices[0]]
            
            # Ground truth for this query.
            gt_set = set(benchmark_data[query_key])
            total_relevant = len(gt_set)
            
            # Dictionary to store metrics for the current query.
            query_metrics = {
                'candidates': retrieved,
                'ground_truth': list(gt_set),
                'precision': [],
                'recall': [],
                'f1': [],
                'ap': []
            }
            
            # Compute metrics at each k.
            for k in range(1, max_k+1):
                retrieved_k = retrieved[:k]
                num_relevant = sum(1 for item in retrieved_k if item in gt_set)
                precision = num_relevant / k
                recall = num_relevant / total_relevant if total_relevant > 0 else 0.0
                f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
                
                # Compute average precision (AP) for top-k.
                ap = 0.0
                relevant_count = 0
                for i, item in enumerate(retrieved_k, start=1):
                    if item in gt_set:
                        relevant_count += 1
                        ap += relevant_count / i
                norm = min(total_relevant, k)
                ap = ap / norm if norm > 0 else 0.0
                
                query_metrics['precision'].append(precision)
                query_metrics['recall'].append(recall)
                query_metrics['f1'].append(f1)
                query_metrics['ap'].append(ap)
                
                # Accumulate system-level metrics.
                system_precision[k-1] += precision
                system_recall[k-1]    += recall
                system_ap[k-1]        += ap
                f1_lists[k-1].append(f1)
            
            # Create a unique key per query (including the split number).
            query_key_str = f"split_{split}_{csv_filename}::{col_name}"
            per_query_metrics[query_key_str] = query_metrics
            total_evaluated_queries += 1
    
    # Average the system metrics over all evaluated queries.
    if total_evaluated_queries > 0:
        avg_system_precision = (system_precision / total_evaluated_queries).tolist()
        avg_system_recall = (system_recall / total_evaluated_queries).tolist()
        avg_system_ap = (system_ap / total_evaluated_queries).tolist()
        avg_system_f1 = [np.mean(f1_list) if f1_list else 0.0 for f1_list in f1_lists]
    else:
        avg_system_precision = [0.0] * max_k
        avg_system_recall = [0.0] * max_k
        avg_system_ap = [0.0] * max_k
        avg_system_f1 = [0.0] * max_k
    
    used_k = list(range(1, max_k+1))
    metrics_at_k = {
        k: {
            'precision': avg_system_precision[k-1],
            'recall': avg_system_recall[k-1],
            'f1': avg_system_f1[k-1],
            'map': avg_system_ap[k-1]
        }
        for k in used_k
    }
    
    results = {
        'system_metrics': {
            'precision': avg_system_precision,
            'recall': avg_system_recall,
            'f1': avg_system_f1,
            'map': avg_system_ap,
            'used_k': used_k,
            'metrics_at_k': metrics_at_k
        },
        'per_query_metrics': per_query_metrics
    }
    
    # Print the aggregated system metrics.
    for k in range(1, max_k+1):
        print(f"k = {k}: Precision@k = {avg_system_precision[k-1]:.3f}, "
              f"Recall@k = {avg_system_recall[k-1]:.3f}, F1@k = {avg_system_f1[k-1]:.3f}, "
              f"MAP@k = {avg_system_ap[k-1]:.3f}")
        
    # if output path doesn't exist, create it
    os.makedirs(os.path.dirname(output_json_file), exist_ok=True)
    
    # Save results to a JSON file.
    with open(output_json_file, "w") as f:
        json.dump(results, f, indent=2)
    
    print(f"Results saved to {output_json_file}")
    return results

In [ ]:
results = evaluate_inference_fasttext_to_json(fasttext_model, index, id_to_metadata, benchmark_data, 
                                              all_tables, n_splits=1, split_frac=1, max_k=10, max_tokens=128, 
                                              output_json_file="output/wiki-join-jaccard/deepjoin/deepjoin_fasttext.json")  